# Import and Libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

RAW_PATH = Path('./Data/Flight_on_time_HIX.csv')
CLEAN_PATH = Path('./Data/Flight_on_time_HIX_cleaned.csv')


In [2]:

df = pd.read_csv(RAW_PATH)
print('DataFrame shape:', df.shape)
df.head(3)


DataFrame shape: (65669, 18)


,Airline,Flight_Number,Plane_ID,FlightDate,Origin_Airport,Destination_Airport,Flight_Distance,Scheduled_Departure_Time,Actual_Departure_Time,Departure_Delay_Minutes,Departure_Taxi,Departure_WheelsOff,Scheduled_Arrival_Time,Actual_Arrival_Time,Arrival_Delay_Minutes,Arrival_Taxi,Arrival_WheelsOn,Delay_Reason
0,TR,4135,GJLOAB,2019-12-01,IYF,HIX,678.0,1,103.0,62.0,15.0,118.0,127,219.0,52.0,7.0,212.0,LateAircraft
1,TR,3776,GDISCI,2019-12-01,HEN,HIX,1700.0,10,25.0,15.0,21.0,46.0,616,624.0,8.0,27.0,557.0,NaN
2,RO,4754,GHHWLL,2019-12-01,HIX,IZN,413.0,10,10.0,0.0,15.0,25.0,242,227.0,0.0,5.0,222.0,NaN


In [3]:

print('\nColumn dtypes:')
print(df.dtypes)
print('\nNull counts:')
print(df.isna().sum())



Column dtypes:
Airline                      object
Flight_Number                 int64
Plane_ID                     object
FlightDate                   object
Origin_Airport               object
Destination_Airport          object
Flight_Distance             float64
Scheduled_Departure_Time      int64
Actual_Departure_Time       float64
Departure_Delay_Minutes     float64
Departure_Taxi              float64
Departure_WheelsOff         float64
Scheduled_Arrival_Time        int64
Actual_Arrival_Time         float64
Arrival_Delay_Minutes       float64
Arrival_Taxi                float64
Arrival_WheelsOn            float64
Delay_Reason                 object
dtype: object

Null counts:
Airline                         0
Flight_Number                   0
Plane_ID                        0
FlightDate                      0
Origin_Airport                  0
Destination_Airport             0
Flight_Distance                 0
Scheduled_Departure_Time        0
Actual_Departure_Time           0
De

# Data Cleaning

The data is in hhmm format (615 -> 06:15). This section builds a few helper functions that convert a variety of different data formats (int, float, etc...) to datetime.time. This is done to ensure consistency in data treatment

In [4]:

import re
from datetime import datetime, timedelta, time

def _coerce_hhmm_to_tuple(x):
    """Return (hour, minute) from flexible inputs:
    - 0-2359 ints/floats representing hhmm (e.g., 5, 45, 615, 1330)
    - 'HH:MM' or 'HH:MM:SS' strings
    - already time/datetime objects
    Returns None if cannot parse.
    """
    if pd.isna(x):
        return None
    # If already datetime.time
    if isinstance(x, time):
        return x.hour, x.minute
    if isinstance(x, datetime):
        return x.hour, x.minute
    # Strings like '08:20:00' or '8:20'
    if isinstance(x, str):
        s = x.strip()
        m = re.match(r'^(\d{1,2}):(\d{2})(?::(\d{2}))?$', s)
        if m:
            hh = int(m.group(1))
            mm = int(m.group(2))
            return hh, mm
        # try numeric string like '820' or '45'
        if s.isdigit():
            try:
                v = int(s)
                hh = v // 100
                mm = v % 100
                return hh, mm
            except:
                return None
        return None
    # Numeric hhmm: 45 -> 00:45, 820 -> 08:20
    if isinstance(x, (int, np.integer)):
        v = int(x)
        hh = v // 100
        mm = v % 100
        return hh, mm
    if isinstance(x, (float, np.floating)):
        v = int(round(x))
        hh = v // 100
        mm = v % 100
        return hh, mm
    return None

def to_clock_str(x):
    t = _coerce_hhmm_to_tuple(x)
    if t is None:
        return np.nan
    hh, mm = t
    # Basic guards
    if not (0 <= hh <= 23 and 0 <= mm <= 59):
        return np.nan
    return f"{hh:02d}:{mm:02d}:00"

def ensure_clock_time(df, col):
    """Coerce a column to 'HH:MM:SS' strings (or NaN)."""
    return df[col].apply(to_clock_str)


Here, scheduled and actual times are coerced (forced) to consistent clock format (hh:mm:ss)

In [5]:
time_cols = [
    'Scheduled_Departure_Time', 'Scheduled_Arrival_Time',
    'Actual_Departure_Time', 'Actual_Arrival_Time',
    'Departure_WheelsOff', 'Arrival_WheelsOn'
]

# Only coerce columns that exist
existing_time_cols = [c for c in time_cols if c in df.columns]
for c in existing_time_cols:
    df[c] = ensure_clock_time(df, c)

df[existing_time_cols].head(5)


,Scheduled_Departure_Time,Scheduled_Arrival_Time,Actual_Departure_Time,Actual_Arrival_Time,Departure_WheelsOff,Arrival_WheelsOn
0,00:01:00,01:27:00,01:03:00,02:19:00,01:18:00,02:12:00
1,00:10:00,06:16:00,00:25:00,06:24:00,00:46:00,05:57:00
2,00:10:00,02:42:00,00:10:00,02:27:00,00:25:00,02:22:00
3,00:10:00,01:11:00,00:44:00,01:55:00,01:18:00,01:51:00
4,00:10:00,01:32:00,02:34:00,03:58:00,03:00:00,03:52:00


Parsing FlightDate column to obtain calendar features (DayOfWeek, Month, etc...). For the specific scope of this case, this step will serve as a proxy for robusting treating delay times

In [6]:

df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
df['DayOfWeek'] = df['FlightDate'].dt.dayofweek    # Monday=0
df['Month'] = df['FlightDate'].dt.month
df['Quarter'] = df['FlightDate'].dt.quarter
df['IsWeekend'] = df['DayOfWeek'].isin([5,6]).astype(int)


Same reasoning as above, deriving hour-of-day features. This may be employed in transversal dataset analysis

In [7]:

def hour_from_clock_str(s):
    if pd.isna(s): 
        return np.nan
    try:
        return int(s[:2])
    except:
        return np.nan

df['Dep_Hour'] = df['Scheduled_Departure_Time'].apply(hour_from_clock_str)
df['Arr_Hour'] = df['Scheduled_Arrival_Time'].apply(hour_from_clock_str)

# Red-eye flights: depart late night or arrive early morning
df['Is_Redeye'] = ((df['Dep_Hour'] >= 22) | (df['Arr_Hour'] <= 5)).astype(int)


Recomputing the delays column, basically ensuring the rollover when the flight is across two days, to avoid having skewed delays result. Without this step, a flight scheduled to leave at 10pm and arrive at 1AM, with a 1 hour delay, would be computed as a 20 hour delay (2:00 - 22:00 -> 20 hr delay)

In [8]:
def combine_date_time(date_series, time_str_series):
    return pd.to_datetime(
        date_series.dt.strftime('%Y-%m-%d') + ' ' + time_str_series,
        errors='coerce'
    )

# Ensuring non-negative delay computation with a simple helper function
def minutes_diff_nonnegative(actual, scheduled):
    if pd.isna(actual) or pd.isna(scheduled):
        return np.nan
    delta = (actual - scheduled).total_seconds() / 60.0
    return max(delta, 0.0)

# Building base datetimes columns, both scheduled and actual
dep_sched_dt = combine_date_time(df['FlightDate'], df['Scheduled_Departure_Time'])
arr_sched_dt = combine_date_time(df['FlightDate'], df['Scheduled_Arrival_Time'])
dep_actual_dt = combine_date_time(df['FlightDate'], df['Actual_Departure_Time'])
arr_actual_dt = combine_date_time(df['FlightDate'], df['Actual_Arrival_Time'])

# Structuring rollover adjustments based on departures, both scheduled and actual
arr_sched_dt_adj = arr_sched_dt.copy()
mask_sched_roll = arr_sched_dt.notna() & dep_sched_dt.notna() & (arr_sched_dt < dep_sched_dt)
arr_sched_dt_adj[mask_sched_roll] = arr_sched_dt_adj[mask_sched_roll] + pd.Timedelta(days=1)

arr_actual_dt_adj = arr_actual_dt.copy()
mask_actual_roll = arr_actual_dt.notna() & dep_actual_dt.notna() & (arr_actual_dt < dep_actual_dt)
arr_actual_dt_adj[mask_actual_roll] = arr_actual_dt_adj[mask_actual_roll] + pd.Timedelta(days=1)

# Recomputing delays with corrected rollovers
df['Departure_Delay_Recomputed'] = [
    minutes_diff_nonnegative(a, s) for a, s in zip(dep_actual_dt, dep_sched_dt)
]
df['Arrival_Delay_Recomputed'] = [
    minutes_diff_nonnegative(a, s) for a, s in zip(arr_actual_dt_adj, arr_sched_dt_adj)
]





Consistency check and diagnostics: comparing computed with provided delays

In [9]:

def compare_delay_cols(provided_col, recomputed_col, tolerance=1.0):
    if provided_col not in df.columns:
        print(f"Provided column '{provided_col}' not present; skipping.")
        return None
    prov = df[provided_col]
    rec = df[recomputed_col]
    both = prov.notna() & rec.notna()
    diffs = (prov[both] - rec[both]).abs()
    exact = (diffs <= tolerance).mean() if both.sum() > 0 else np.nan
    print(f"Rows with both values: {both.sum():,}")
    print(f"Share within ±{tolerance} minute: {exact:.3f}")
    print("Mean absolute difference (mins):", round(diffs.mean(), 3) if both.sum() else 'NA')
    return diffs

print('Departure delays:')
_ = compare_delay_cols('Departure_Delay_Minutes', 'Departure_Delay_Recomputed', tolerance=1.0)

print('\nArrival delays:')
_ = compare_delay_cols('Arrival_Delay_Minutes', 'Arrival_Delay_Recomputed', tolerance=1.0)


Departure delays:
Rows with both values: 64,667
Share within ±1.0 minute: 0.996
Mean absolute difference (mins): 2.041

Arrival delays:
Rows with both values: 64,653
Share within ±1.0 minute: 0.988
Mean absolute difference (mins): 327.216


In [10]:
# Adjusting block times (gate-out to gate-in) with newly computed adjusted rollovers
sched_block = arr_sched_dt_adj - dep_sched_dt
act_block = arr_actual_dt_adj - dep_actual_dt

df['Scheduled_BlockTime_Min'] = sched_block.dt.total_seconds() / 60.0
df['Actual_BlockTime_Min'] = act_block.dt.total_seconds() / 60.0

# Segmenting block time into taxi and airborne components
if {'Departure_Taxi','Arrival_Taxi'}.issubset(df.columns):
    df['Total_Taxi_Time_Min'] = df['Departure_Taxi'].fillna(0) + df['Arrival_Taxi'].fillna(0)
else:
    df['Total_Taxi_Time_Min'] = np.nan

df['Airborne_Time_Min'] = df['Actual_BlockTime_Min'] - df['Total_Taxi_Time_Min']

# Creating binary flags for departure/arrival delays over 15 minutes
df['Is_Departure_Delayed_15'] = (df['Departure_Delay_Recomputed'] > 15).astype('Int64')
df['Is_Arrival_Delayed_15'] = (df['Arrival_Delay_Recomputed'] > 15).astype('Int64')


Sanity checks, to ensure that there has been no major error in treating the data

In [11]:

summary = {
    'rows': len(df),
    'n_airlines': df['Airline'].nunique() if 'Airline' in df.columns else None,
    'n_routes': df['Route'].nunique() if 'Route' in df.columns else None,
    'dep_delay_mean': round(df['Departure_Delay_Recomputed'].mean(), 2),
    'arr_delay_mean': round(df['Arrival_Delay_Recomputed'].mean(), 2),
    'pct_dep_delayed_15': round(100 * (df['Is_Departure_Delayed_15'] == 1).mean(), 2),
    'pct_arr_delayed_15': round(100 * (df['Is_Arrival_Delayed_15'] == 1).mean(), 2),
}
summary


{'rows': 65669,
 'n_airlines': 16,
 'n_routes': None,
 'dep_delay_mean': np.float64(14.24),
 'arr_delay_mean': np.float64(26.04),
 'pct_dep_delayed_15': np.float64(18.29),
 'pct_arr_delayed_15': np.float64(20.67)}

In [12]:

mv = df.isna().sum().sort_values(ascending=False)
mv.head(15)


Delay_Reason                  51588
Airborne_Time_Min              1018
Actual_BlockTime_Min           1018
Arrival_Delay_Recomputed       1016
Departure_Delay_Recomputed     1002
DayOfWeek                      1000
Quarter                        1000
Month                          1000
Scheduled_BlockTime_Min        1000
FlightDate                     1000
Arrival_WheelsOn                 29
Actual_Arrival_Time              17
Actual_Departure_Time             2
Departure_WheelsOff               2
Total_Taxi_Time_Min               0
dtype: int64

Last step: saving cleaned data to our data folder. We will use the cleaned df from now on

In [13]:
# Save cleaned total data
df.to_csv(CLEAN_PATH, index=False)


# Data Filtering

Re-importing the clean dataset. This step can be skipped if the whole notebook is ran from the top, but in case only the following piece was ran, this code piece ensures consistency

In [ ]:
df_clean = pd.read_csv(CLEAN_PATH)
if "FlightDate" in df_clean.columns:
    df_clean["FlightDate"] = pd.to_datetime(df_clean["FlightDate"], errors="coerce")

print(f"Loaded clean dataset from: {CLEAN_PATH}  |  rows={len(df_clean):,}")


At this point, the clean dataset can be filtered to keep only flghts from HIX operated by CA

In [ ]:
# Filter to Compass Airlines (CA) flights departing from HIX
required_cols = [
    "Airline","Origin_Airport","Destination_Airport","FlightDate",
    "Dep_Hour","Arr_Hour",
    "Is_Arrival_Delayed_15",
    "Arrival_Delay_Recomputed","Departure_Delay_Recomputed","Total_Taxi_Time_Min"
]
missing = [c for c in required_cols if c not in df_clean.columns]
if missing:
    raise KeyError(f"Missing required columns for CA/HIX analysis: {missing}")

df_ca_hix = df_clean.query("Airline == 'CA' and Origin_Airport == 'HIX'").copy()
print(f"Filtered flights from HIX and operated by CA dataset rows = {len(df_ca_hix):,}")

CA_HIX_PATH = Path("./Data/Flight_on_time_HIX_CA.csv")
df_ca_hix.to_csv(CA_HIX_PATH, index=False)
print(f"Saved filtered dataset → {CA_HIX_PATH.resolve()}")


# KPI and insights from data

Identifying a non-exahustive listing of KPIs. These should give us a decent overlook of some significant trend in the data, mainly by cross-referencing the time that takes to carry out different flight steps (taxi time vs delay, for instance)

In [16]:
def pct(x):
    return None if pd.isna(x) else f"{round(100*x,2)}%"

if len(df_ca_hix) == 0:
    raise ValueError("No CA flights departing HIX found. Check filters or data period.")

kpis = {
    "Rows (CA from HIX)": len(df_ca_hix),
    "Unique destinations": df_ca_hix["Destination_Airport"].nunique(),
    "Arrival on-time rate (<15m)": pct(1 - df_ca_hix["Is_Arrival_Delayed_15"].mean()),
    "Avg arrival delay (min)": round(df_ca_hix["Arrival_Delay_Recomputed"].mean(), 2),
    "Avg departure delay (min)": round(df_ca_hix["Departure_Delay_Recomputed"].mean(), 2),
    "Avg total taxi time (min)": round(df_ca_hix["Total_Taxi_Time_Min"].mean(), 2),
}
kpi_df = pd.DataFrame(list(kpis.items()), columns=["Metric","Value"])
kpi_df


,Metric,Value
0,Rows (CA from HIX),5256
1,Unique destinations,117
2,Arrival on-time rate (<15m),77.09%
3,Avg arrival delay (min),15.99
4,Avg departure delay (min),15.72
5,Avg total taxi time (min),31.74


Here, another possible approach: looking from the route-level point of view, by taking into consideration the first 15 destinations by destination airport

In [17]:
route_summary = (
    df_ca_hix
    .groupby("Destination_Airport")
    .agg(
        flights=("Is_Arrival_Delayed_15","size"),
        on_time=("Is_Arrival_Delayed_15", lambda s: 1 - s.mean()),
        avg_arr_delay=("Arrival_Delay_Recomputed","mean"),
        avg_dep_delay=("Departure_Delay_Recomputed","mean"),
        avg_taxi=("Total_Taxi_Time_Min","mean"),
    )
    .sort_values("flights", ascending=False)
    .head(15)
    .reset_index()
)
route_summary["on_time"] = (route_summary["on_time"]*100).round(1).astype(str) + "%"
route_summary.round({"avg_arr_delay":2,"avg_dep_delay":2,"avg_taxi":2})


,Destination_Airport,flights,on_time,avg_arr_delay,avg_dep_delay,avg_taxi
0,UEU,286,81.5%,14.58,13.87,33.40
1,WKQ,200,79.0%,13.97,13.64,34.90
2,FBY,135,77.0%,15.42,13.53,33.73
3,YNU,132,76.5%,19.02,17.49,33.56
4,HBW,128,71.1%,16.16,15.66,31.67
5,BEX,128,74.2%,15.50,13.34,35.73
6,LPL,119,84.9%,13.03,13.35,27.52
7,LCW,108,74.1%,21.46,24.99,33.55
8,WJG,103,74.8%,16.06,15.57,32.39
9,TJY,99,70.7%,27.65,30.53,31.18


In [19]:
import plotly.express as px
import plotly.graph_objects as go

In [27]:
# On-time rate by destination, sorted
ontime_dest = (1 - df_ca_hix.groupby("Destination_Airport")["Is_Arrival_Delayed_15"].mean())\
                .rename("on_time_rate").reset_index().sort_values("on_time_rate", ascending=False)

fig1 = px.bar(
    ontime_dest,
    x="Destination_Airport",
    y="on_time_rate",
    template= "plotly_dark",
    title="CA from HIX — Arrival On-Time Rate by Destination",
)
fig1.update_yaxes(range=[0,1], title="On-Time Rate")
fig1.update_xaxes(title="Destination")
fig1.update_layout(margin=dict(l=20,r=20,t=60,b=20))
fig1.show()

In [28]:
risk_by_hour = df_ca_hix.groupby("Dep_Hour")["Is_Arrival_Delayed_15"].mean()\
                .reindex(range(24)).reset_index().rename(columns={"Dep_Hour":"dep_hour","Is_Arrival_Delayed_15":"delay_share"})

fig2 = px.line(
    risk_by_hour,
    x="dep_hour",
    y="delay_share",
    markers=True,
    template= "plotly_dark",
    title="CA from HIX — Arrival Delay Risk by Departure Hour (≥15m)",
)
fig2.update_yaxes(range=[0,1], title="Share Delayed (≥15m)")
fig2.update_xaxes(title="Departure Hour (0–23)", dtick=1)
fig2.update_layout(margin=dict(l=20,r=20,t=60,b=20))
fig2.show()

In [29]:
if "FlightDate" in df_ca_hix.columns and df_ca_hix["FlightDate"].notna().any():
    daily_ontime = (1 - df_ca_hix.groupby(df_ca_hix["FlightDate"].dt.day)["Is_Arrival_Delayed_15"].mean())\
                    .reset_index().rename(columns={"FlightDate":"day","Is_Arrival_Delayed_15":"on_time_rate"})
    daily_ontime.columns = ["day", "on_time_rate"]

    fig3 = px.line(
        daily_ontime,
        x="day",
        y="on_time_rate",
        markers=True,
        template= "plotly_dark",
        title="CA from HIX — Daily Arrival On-Time Rate (Dec 2019)",
    )
    fig3.update_yaxes(range=[0,1], title="On-Time Rate")
    fig3.update_xaxes(title="Day of Month", dtick=1)
    fig3.update_layout(margin=dict(l=20,r=20,t=60,b=20))
    fig3.show()



In [31]:
fig4 = px.scatter(
    df_ca_hix,
    x="Total_Taxi_Time_Min",
    y="Arrival_Delay_Recomputed",
    opacity=0.5,
    trendline=None,
    title="CA from HIX — Total Taxi Time vs Arrival Delay",
    template= "plotly_dark",
    labels={"Total_Taxi_Time_Min":"Total Taxi Time (min)", "Arrival_Delay_Recomputed":"Arrival Delay (min)"}
)
fig4.update_layout(margin=dict(l=20,r=20,t=60,b=20))
fig4.show()
